# EcoInvest Stock Trading Dashboard 

- __Name__: Prathiksha Ramaswamy
- __Student ID__: 503411pr

## Importing the Necessary Libraries & Market Data

In [2]:
# General purpose and Data Manipulation
import numpy as np
import pandas as pd

# (Financial) Data Visualization 
import datetime as dt 
import plotly
import plotly.graph_objs as go 
from pandas_datareader import data as pdr
from ta.trend import MACD
from ta.momentum import StochasticOscillator
import matplotlib as mpl

# Web Application Development, Dash, Dependencies
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc 
from dash import Dash, html
import base64
from PIL import Image

# KPIs for Market Data (Yahoo Finance Website)
import yfinance as yf
import yahooquery as yq;

## Backend Features and How they are Connected

The backend software of the Stock Trading App are robot advisors that recommend which stocks to buy based on users' preferences, share market data and business performance. These also handle the trading strategies through which shares are bought and sold. The robot advisors' recommendation signals guide traders on where to start as a beginner. The GUI helps users interact with the vast data visualizations and grasp how companies are valued and assessed before buying their stocks. It also facilitates the search of stock market data on any company they are interested in. A lot of information is accessible to navigate all in one place *and* the visuals are easy to read.

## Accessing Stocks Data from Yahoo Finance & Yahoo Query

After playing around with different companies' information to visualize, it was evident that data was missing or inconsistent for many companies, namely sustainability ones. As these are essential to EcoInvest, I conducted research on some sustainable companies and added them manually to the dashboard. To scale up the amount of companies information accessible on the financial assistant tab and whole app in general, a dataset with sustainable companies would be handy to source the financial data from. However, at this stage, a subset of sustainable companies is presented. 

Originally, the sought after API to access the financial data was Yahoo Finance. However, due to some bugs that are still under maintenance in the library, a lot of the financial statements are still inaccessible. An alternative API with almost identical features is Yahoo Query. A combination of both are used since neither had complete data. 

In [3]:
# The ticker symbols as well as labels for the selected stocks are stored in a list to access later
stockses = ['BN.PA','SDAC','ENPH','CLH','WY']
names = ['Danone', 'Sustainable Development Acquisition I Corp.','Enphase Energy, Inc.', 'Clean Harbors, Inc.','Weyerhaeuser Company']

# Empty dictionaries to store the accessed data to manipulate later
dict_income_st = {}
dict_balance_sheet = {}
dict_price = {}
dict_info = {}

# Downloading data from Yahoo Finance and Yahoo Query
# Accessing balance sheet, income statement, historical share prices, overview of all current financial metrics
for ticker in stockses:
    dict_balance_sheet[ticker] = pd.DataFrame(yq.Ticker('{}'.format(ticker)).balance_sheet()) 
    dict_balance_sheet[ticker].set_index('asOfDate', inplace = True)
    dict_income_st[ticker]= pd.DataFrame(yq.Ticker('{}'.format(ticker)).income_statement(frequency = 'q'))
    dict_income_st[ticker].set_index('asOfDate', inplace = True)
    dict_income_st[ticker]['Net Profit Margin'] = dict_income_st[ticker]['NetIncome']/dict_income_st[ticker]['TotalRevenue']*100
    dict_price[ticker] = pd.DataFrame(yf.Ticker('{}'.format(ticker)).history(period="2y", interval = "1d"))
    dict_info[ticker]  = pd.DataFrame(yf.Ticker('{}'.format(ticker)).info)

# Storing tickers and labels in a dataframe to manipulate later for the dashboard
p= pd.DataFrame(stockses)
d = pd.DataFrame(names)
pp_labels = [{'label': d[0].unique()[i], 'value': p[0].unique()[i]} for i in range(p[0].unique().shape[0])]
pp_labels

[{'label': 'Danone', 'value': 'BN.PA'},
 {'label': 'Sustainable Development Acquisition I Corp.', 'value': 'SDAC'},
 {'label': 'Enphase Energy, Inc.', 'value': 'ENPH'},
 {'label': 'Clean Harbors, Inc.', 'value': 'CLH'},
 {'label': 'Weyerhaeuser Company', 'value': 'WY'}]

# Creating the Dash App

The main feature and UX of the dashboard is the great ease of use and navigation between company stock market data and business performance data. It is aimed at beginner's stock traders, specifically for sustainable stocks aimed at increasing investment in the UN's SDGs.  

The idea is to create a webpage with 3 tabs with the below layout. The tabs have the following functions: 
- __Tab1__ : Get stock market information on top companies from which user has bought stocks, in terms of share price. If new, these will just show the top 3 stocks in general. This page is to include focussed details about the securities'. Including the price, the volume, moving average signals and stochastic signals. All these terms and indicators will be both displayed and described to get a meaningful understanding of the security. It is an overview of current stocks held by user, to monitor and compare the activities. The user can scroll through the top stocks using a dropdown menu, which will trigger a callback to adjust the information displayed accordingly.


- __Tab2__: The second tab is a financial assistant board for the businesses themselves. This will illustrate the main financials of the company selected and depict the recent evolution of the company's performance as these are also important indicators of the stock price momentum. These figures are also to be explained on the bottom of the page to support the users interpreting them. Again, the sought company can be navigated through a dropdown menu.


- __Tab3__: The final tab serves as a data query functionality. It allows the user to search up any stocks, whether sustainable or not and whether they invest in them or not (in the case that they are sustainable). Here the trader can use a input search box to access the data. The graphs displayed on this page are the same as those on tab1.  

In [4]:
# Creating the app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

# Creating the various Dash/HTML components
# Defining the content to add to Tab 1: Overview of Top 3 Stocks in terms of Share Price User has invested in
# Callback and function for live Stock market figures & Dropdown Menu to scroll various stocks
@app.callback(Output(component_id='line_plot', component_property= 'figure', allow_duplicate = True),
              [Input(component_id='dropdown', component_property= 'value')], prevent_initial_call = True)

def create_graph(dropdown_value):
    df=yf.download(tickers='{}'.format(dropdown_value),period='1d',interval='1m')
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()

    # MACD
    macd = MACD(close=df['Close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)

    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                                 close=df['Close'],
                                 low=df['Low'],
                                 window=14, 
                                 smooth_window=3)
    fig=go.Figure()

    # add subplot properties when initializing fig variable
    fig = plotly.subplots.make_subplots(rows=4, cols=1, shared_xaxes=True,
                        vertical_spacing=0.01, 
                        row_heights=[0.5,0.1,0.2,0.2])

    fig.add_trace(go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'], name = 'market data'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['MA5'], 
                             opacity=0.7, 
                             line=dict(color='blue', width=2), 
                             name='MA 5'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['MA20'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='MA 20'))
    


    # Plot volume trace on 2nd row
    colors = ['green' if row['Open'] - row['Close'] >= 0 
              else 'red' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(x=df.index, 
                         y=df['Volume'],
                         marker_color=colors, name = 'Volume'
                        ), row=2, col=1)

    
    # Plot MACD trace on 3rd row
    colorsM = ['green' if val >= 0 
              else 'red' for val in macd.macd_diff()]
    fig.add_trace(go.Bar(x=df.index, 
                         y=macd.macd_diff(),
                         marker_color=colorsM
                        ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=macd.macd(),
                             line=dict(color='black', width=2)
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=macd.macd_signal(),
                             line=dict(color='blue', width=1)
                            ), row=3, col=1)

    # Plot stochastics trace on 4th row
    fig.add_trace(go.Scatter(x=df.index,
                             y=stoch.stoch(),
                             line=dict(color='black', width=2), name = 'Stoch'
                            ), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=stoch.stoch_signal(),
                             line=dict(color='blue', width=1), name = 'Stoch Signal'
                            ), row=4, col=1)

    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    labels_to_show_in_legend = ["MA 5", "MA 20", "Volume",'market data', 'Stoch', 'Stoch Signal']
    fig.update_layout(height=900, width=1200, 
                      showlegend=True, 
                      xaxis_rangeslider_visible=False)
    for trace in fig['data']: 
        if (not trace['name'] in labels_to_show_in_legend):
            trace['showlegend'] = False

    # Make the title dynamic to reflect whichever stock we are analyzing
    fig.update_layout(
    title= str('{}'.format(dropdown_value))+' Live Share Price: '  ,
    yaxis_title='Stock Price (USD per Shares)', template = "plotly_dark") 

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
    fig.update_yaxes(title_text="Stoch", row=4, col=1)           

    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector_visible=False,
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="HTD", step="hour", stepmode="todate"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    return fig  


In [5]:
# Callback and function for text describing live price changes
@app.callback(Output('output_change', 'children'), [Input('dropdown', 'value')])
def changes(dropdown_value):
    tick = yf.Ticker('{}'.format(dropdown_value))
    previous = tick.info['previousClose']
    current = tick.info['currentPrice']
    change = (current-previous)/previous*100
    final_change = '{:.2f}'.format(change)
    return f"The price of {dropdown_value}'s stock has changed by {final_change}% since the previous close"

In [13]:
# Tab 1 Layout
tab1 = html.Div(className = "graphtest", children = [
    html.H3(id = 'H1', children = 'Overview of your top stocks', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40, 'color':'#54FF9F'}),

        dcc.Dropdown( id = 'dropdown',
        options = [
            {'label':'Danone', 'value':'BN.PA' },
            {'label': 'Coloplast A/S', 'value':'COLO-B.CO'},
            {'label': 'Planet Labs', 'value':'PL'}
            ],
        value = 'COUR',style={'backgroundColor':'#111111', 'color':'green'}), dcc.Markdown(''' **Price:** The first plot illustrates the live stock price in USD $ per Share

**Volume:** The Volume represents the number of shares traded during a given period. This can be used to indicate interest of securities. Increased volume, and therefore interest, signals that security is gaining attention and that trading activity is increasing herein. 

**MACD:** The Moving Average Convergence Divergence (MACD) indicator demonstrates the direction of trend and the strength between two moving averages of a share’s price. This instrument can be used to determine under/over-valuation and subsequently buy/sell decisions, depending on the momentum of the share price. In this plot, you can see the 5-day MA and 20-day MA.  

**Stochastic Oscillator:** The Stochastic Oscillator displays the momentum of a security by comparing its closing price to the stock price's high and low range over a certain period of time. It can be used to determine trends and predict reversals.  ''', style = {'color':'white'}) , html.Div(id = 'output_change', style = {'color':'turquoise'}) ,      dcc.Graph(id = 'line_plot'),
    ] )


In [6]:
# Content to add to Tab 2: Financial Assistant Dashboard with financials data on selected companies
# Callback and function to create financials visualizations
# The included plots: Bar plots for revenue, net profit margin, shares outstanding, net income, total liabilities,
# Cash and Cash Equivalents and a line chart of 2 years share price history
# Some metrics displayed include: P/E ratio, latest price and also sector & industry classifications
@app.callback(
    [
        Output("revenuechart", "figure"), Output("netprofitmargchart", "figure"), Output("shareschart", "figure"),
        Output("ebitdachart", "figure"), Output("debtchart", "figure"), Output("freecashchart", "figure"),
        Output("pricechart", "figure"), Output("sec", "children"), Output("ind", "children"),
        Output("p/e", "children"), Output("comp", "children"), Output("pri", "children")
    ],
    Input('stock-dropdown', 'value')
)

def update_figure(stockses):    
    THEME='ggplot2'
    MARGIN=dict(t=10, b=20, l=10, r=10)
    
    fig_rev = px.bar(dict_income_st[stockses]['TotalRevenue'], y='TotalRevenue',labels = {'asOfDate':'Date'}, text_auto='.3s', hover_data = {'TotalRevenue': ':,.2d'})
    fig_rev.update_layout(template="plotly_dark", margin = MARGIN)
    fig_rev.update_traces(textposition="outside", cliponaxis=False, marker_color='mediumaquamarine')
    
    fig_net_inc = px.bar(dict_income_st[stockses]['NetIncome'], y='NetIncome', labels = {'asOfDate':'Date'},text_auto='.3s', hover_data = {'NetIncome': ':,.2d'})
    fig_net_inc.update_layout(template='plotly_dark', margin = MARGIN)
    fig_net_inc.update_traces(marker_color='skyblue', textposition="outside", cliponaxis=True)
    
    fig_debt = px.bar(dict_balance_sheet[stockses]['TotalLiabilitiesNetMinorityInterest'], y='TotalLiabilitiesNetMinorityInterest',labels = {'TotalLiabilitiesNetMinorityInterest':'Total Liabilities', 'asOfDate':'Date'}, text_auto='.3s', hover_data = {'TotalLiabilitiesNetMinorityInterest': ':,.2d'})
    fig_debt.update_layout(template='plotly_dark', margin = MARGIN)
    fig_debt.update_traces(marker_color='peachpuff', textposition="outside", cliponaxis=False)
    
    fig_fcf = px.bar(dict_balance_sheet[stockses]['CashAndCashEquivalents'], y='CashAndCashEquivalents',labels = {'asOfDate':'Date'}, text_auto='.3s', hover_data = {'CashAndCashEquivalents': ':,.2d'})
    fig_fcf.update_layout(template='plotly_dark', margin = MARGIN)
    fig_fcf.update_traces(marker_color='darkturquoise', textposition="outside", cliponaxis=False)

    fig_shares = px.bar(dict_balance_sheet[stockses]['CapitalStock'], y='CapitalStock',labels = {'CapitalStock':'Shares Outstanding','asOfDate':'Date'}, text_auto='.3s', hover_data = {'CapitalStock': ':,.2d'})
    fig_shares.update_layout(template='plotly_dark', margin = MARGIN)
    fig_shares.update_traces(marker_color='lightsalmon', textposition="outside", cliponaxis=False)

    fig_netprof = px.bar(dict_income_st[stockses]['Net Profit Margin'], y='Net Profit Margin',labels = {'asOfDate':'Date'}, text_auto='.2f')
    fig_netprof.update_layout(template='plotly_dark', margin = MARGIN)
    fig_netprof.update_traces(marker_color='darkkhaki', textposition="outside", cliponaxis=False)

    fig_pricehist = px.line(dict_price[stockses], y='Close')
    fig_pricehist.update_layout(template='plotly_dark', margin = MARGIN)
    fig_pricehist.update_traces(line_color='royalblue', line_width=3, hovertemplate='Close @ %{x}: $%{y}')
    
    sec = 'Sector: ' + dict_info[stockses]['sector'].iloc[0]
    ind = 'Industry: ' + dict_info[stockses]['industry'].iloc[0]
    pe = 'P/E Ratio: ' + str(dict_info[stockses]['trailingPE'].iloc[0])
    comp = 'Company Name: ' + dict_info[stockses]['shortName'].iloc[0]
    pri = 'Latest Price: ' + str(dict_info[stockses]['currentPrice'].iloc[0])

    
    return fig_rev, fig_netprof, fig_shares, fig_net_inc, fig_debt, fig_fcf, fig_pricehist, sec, ind, pe, comp, pri

In [16]:
# Tab 2 Layout

# Loading EcoInvest Logo to add to Web App later
image_path = 'assets/ecoinvest_icon.png'

#Using Pillow to read the the image
pil_img = Image.open("assets/ecoinvest_icon.png")

# Using base64 encoding and decoding
def b64_image(image_filename):
    with open(image_filename, 'rb') as f:
        image = f.read()
    return 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')

tab2  = [
    dbc.Row([
        dbc.Col([
            html.H3('Key Performance Indicators', className='text-center mb-3 p-3', style = {'color':'#54FF9F'}),
            html.Hr(),
        ],
            width={'size': 12, 'offset': 0, 'order': 0}),
    ]),
    dbc.Row([
        dbc.Col([
            html.H6('Select the stock you want to look into:', className='text-center mb-2 p-1', style = {'color':'white'}),
            dcc.Dropdown(
                id = 'stock-dropdown',
                options = pp_labels,
                value = pp_labels[0]['label'],
                clearable=False,
            ),
            html.Div(id='comp', className='text-center mt-3 p-2', style = {'color':'#C0FF3E'}),
            html.Div(id='pri', className='text-center p-2', style = {'color':'#C0FF3E'}),
            html.Div(id='sec', className='text-center p-2', style = {'color':'#C0FF3E'}),
            html.Div(id='ind', className='text-center p-2', style = {'color':'#C0FF3E'}),
            html.Div(id='p/e', className='text-center p-2', style = {'color':'#C0FF3E'})

        ],
            width={'size': 4, 'offset': 0, 'order': 0}),

        dbc.Col([
            html.H5('2 Year Price History', className='text-center p-1', style = {'color':'white'}),
            dcc.Graph(id='pricechart', style={'height': 400}),
        ],
            width={'size': 8, 'offset': 0, 'order': 0}),
    ]),
    dbc.Row([
        dbc.Col([
            html.H5('TotalRevenue', className='text-center', style = {'color':'white'}),
            html.H6('$', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='revenuechart', style={'height': 350}),
            html.H5('Net Profit Margin', className='text-center', style = {'color':'white'}),
            html.H6('%', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='netprofitmargchart', style={'height': 350}),
        ],
            width={'size': 4, 'offset': 0, 'order': 0}),
        dbc.Col([
            html.H5('Net Income', className='text-center', style = {'color':'white'}),
            html.H6('$', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='ebitdachart', style={'height': 350}),
            html.H5('Cash and Cash Equivalents', className='text-center', style = {'color':'white'}),
            html.H6('$', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='freecashchart', style={'height': 350}),
        ],
            width={'size': 4, 'offset': 0, 'order': 0}),
        dbc.Col([
            html.H5('Total Liabilities', className='text-center', style = {'color':'white'}),
            html.H6('$', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='debtchart', style={'height': 350}),
            html.H5('Shares Outstanding', className='text-center', style = {'color':'white'}),
            html.H6('', className='text-center font-italic', style = {'color':'white'}),
            dcc.Graph(id='shareschart', style={'height': 350}),
        ],
            width={'size': 4, 'offset': 0, 'order': 0}),
    ]),dcc.Markdown(''' Glossary
    
'G' stands for giga (billions)

'M' stands for mega (millions)

Revenue: The Gross Income, how much money company has generated in given period.

Net Income: Revenue less the operating expenses.

Total Liabilities: What the company owes.

Net profit margin: Measures how much net income is generated from revenue. The higher this figure, the higher the company is able to generate profit and cover costs. 

Cash and Cash Equivalents: Refer to company’s assets that are cash or that can be converted to cash immediately. This is important to overcome financial crises. 

Shares outstanding: Are the company’s stock currently held by all its shareholders and therefore the amount of stock on the open market. 

The Price-to-Earnings Ratio: The company's share price over its earning per share (EPS). If a company's share price is high compared to its EPS, it may be overvalued. If it is low compared to its EPS, it may be undervalued.
  ''', style = {'color':'white'})
]

yf.pdr_override()

In [9]:
# Tab 3 Content, Live Stock Market plots of any searched company via input search box
# Callback and function
@app.callback(Output(component_id='line_chart', component_property= 'figure'),
              Input(component_id='input', component_property= 'value'))

def output_text(value): # same plots as from tab 1
    df=yf.download(tickers='{}'.format(value),period='1d',interval='1m')
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()

    # MACD
    macd = MACD(close=df['Close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)

    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                                 close=df['Close'],
                                 low=df['Low'],
                                 window=14, 
                                 smooth_window=3)
    fig=go.Figure()

    # add subplot properties when initializing fig variable
    fig = plotly.subplots.make_subplots(rows=4, cols=1, shared_xaxes=True,
                        vertical_spacing=0.01, 
                        row_heights=[0.5,0.1,0.2,0.2])

    fig.add_trace(go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'], name = 'market data'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['MA5'], 
                             opacity=0.7, 
                             line=dict(color='blue', width=2), 
                             name='MA 5'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['MA20'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='MA 20'))

    # Plot volume trace on 2nd row
    colors = ['green' if row['Open'] - row['Close'] >= 0 
              else 'red' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(x=df.index, 
                         y=df['Volume'],
                         marker_color=colors, name = "Volume"
                        ), row=2, col=1)

    # Plot MACD trace on 3rd row
    colorsM = ['green' if val >= 0 
              else 'red' for val in macd.macd_diff()]
    fig.add_trace(go.Bar(x=df.index, 
                         y=macd.macd_diff(),
                         marker_color=colorsM
                        ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=macd.macd(),
                             line=dict(color='black', width=2)
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=macd.macd_signal(),
                             line=dict(color='blue', width=1)
                            ), row=3, col=1)

    # Plot stochastics trace on 4th row
    fig.add_trace(go.Scatter(x=df.index,
                             y=stoch.stoch(),
                             line=dict(color='black', width=2), name = 'Stoch'
                            ), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                             y=stoch.stoch_signal(),
                             line=dict(color='blue', width=1), name = 'Stoch Signal'
                            ), row=4, col=1)

    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    labels_to_show_in_legend = ["MA 5", "MA 20", "Volume",'market data', 'Stoch', 'Stoch Signal']
    fig.update_layout(height=900, width=1200, 
                      showlegend=True, 
                      xaxis_rangeslider_visible=False)
    for trace in fig['data']: 
        if (not trace['name'] in labels_to_show_in_legend):
            trace['showlegend'] = False

    # Make the title dynamic to reflect whichever stock we are analyzing
    tick = yf.Ticker('{}'.format(value))
    previous = tick.info['previousClose']
    current = tick.info['currentPrice']
    change = (current-previous)/previous*100
    final_change = '{:.2f}'.format(change)
    
    fig.update_layout(
        title= str('{}'.format(value))+' Live Share Price:' + final_change + '%',
        yaxis_title='Stock Price (USD per Shares)', template = "plotly_dark", showlegend = True) 
    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
    fig.update_yaxes(title_text="Stoch", row=4, col=1)           

    fig.update_xaxes(
        rangeslider_visible=False,
        rangeselector_visible=False,
        rangeselector=dict(
            buttons=list([
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=45, label="45m", step="minute", stepmode="backward"),
                dict(count=1, label="HTD", step="hour", stepmode="todate"),
                dict(count=3, label="3h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    return fig 

In [10]:
# Tab 3 Layout
tab3 = html.Div(id = 'parent', children = [
    html.H3(id = 'H1', children = 'Overview of stock', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40, 'color':'#C0FF3E'}),dbc.Input(id = "input",
            type='text',
            placeholder="Search...",style={'backgroundColor':'#111111', 'color':'#C0FF3E'}),dcc.Graph(id = 'line_chart')
            ]
               )

In [11]:
# Some formatting, picking colors for various webpage elements
colors = {
    'background': '#1A1A1A',
    'text': '#54FF9F'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor':"#54FF9F",
    'color': 'white',
    'padding': '6px'
}

In [18]:
# Layout of Entire App, adding all the tabs to the webpage and headings
app.layout = html.Div(
    [html.H1('EcoInvest', style={
            'color': colors['text']
        }),
     html.Img(src=image_path, style={'height':'5%', 'width':'5%','align': 'center'}), # Adding image of EcoInvest Logo
    dcc.Tabs(id='tabs', value='1', children=[
        dcc.Tab(
            label='Your Overview',
            value='1',
            children=tab1,selected_style=tab_selected_style 
        ),
        dcc.Tab(
            label='Key Performance Indicators',
            value='2',
            children=tab2,selected_style=tab_selected_style 
        )
    , dcc.Tab(label = 'Search Stocks',
               value = '3',
               children = tab3, selected_style=tab_selected_style)])
]+ [html.Div(id="out-all-types")],style={'backgroundColor': colors['background']})


## Running the Dashboard Web App

In [23]:
# Running the App via local server
if __name__ == '__main__': 
    app.run_server();


Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NONE: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## References Used in Developing this GUI


https://medium.com/@jsteinb/python-adding-features-to-your-stock-market-dashboard-with-plotly-4208d8bc3bd5

https://www.investopedia.com/terms/m/macd.asp

https://www.investopedia.com/

https://github.com/fnneves/stockfinancials_overview/blob/main/Stocks%20Overview%20Dashboard.ipynb

https://medium.com/@jsteinb/python-adding-features-to-your-stock-market-dashboard-with-plotly-4208d8bc3bd5